We need to downsample based on cell counts and a given read depth.

In [60]:
import pandas as pd
import glob
import os

In [61]:
df_stats_full = pd.read_csv('../general/public.general_statistics.tsv', sep='\t', index_col=0)

In [62]:
df_stats_full

,centre,technology,reads,cells,RPC,nreads,%_correct_barcodes,r1_length,r2_length,avg_insert_size,%_mapq30,avg_map_quality
BIO_ddseq_m1c1,BioRad,BioRad ddSeq SureCell ATAC,105603526.0,4253,24830.361157,105603526.0,97.13,54.0,40.0,169.0,93.28,32.9
BIO_ddseq_m1c2,BioRad,BioRad ddSeq SureCell ATAC,141046485.0,4687,30093.126734,141046485.0,97.20,54.0,40.0,168.0,93.17,33.0
BIO_ddseq_m1c3,BioRad,BioRad ddSeq SureCell ATAC,133577432.0,4395,30393.044824,133577432.0,97.17,54.0,40.0,171.0,93.43,33.0
BIO_ddseq_m1c4,BioRad,BioRad ddSeq SureCell ATAC,133016447.0,4372,30424.621912,133016447.0,97.17,54.0,40.0,168.0,93.55,33.0
BIO_ddseq_m1c5,BioRad,BioRad ddSeq SureCell ATAC,187686881.0,4538,41358.942486,187686881.0,97.04,54.0,40.0,170.0,93.22,33.1
BIO_ddseq_m1c6,BioRad,BioRad ddSeq SureCell ATAC,123828247.0,4579,27042.639659,123828247.0,97.25,54.0,40.0,172.0,93.59,33.1
BIO_ddseq_m1c7,BioRad,BioRad ddSeq SureCell ATAC,107051140.0,3726,28730.848094,107051140.0,97.17,54.0,40.0,173.0,93.49,33.0
BIO_ddseq_m1c8,BioRad,BioRad ddSeq SureCell ATAC,129543060.0,3273,39579.303391,129543060.0,97.17,54.0,40.0,172.0,93.42,33.1
BIO_ddseq_m2c1,BioRad,BioRad ddSeq SureCell ATAC,199963379.0,2536,78849.912855,199963379.0,95.75,54.0,40.0,132.0,90.45,31.0
BIO_ddseq_m2c2,BioRad,BioRad ddSeq SureCell ATAC,146879465.0,3456,42499.845197,146879465.0,95.80,54.0,40.0,144.0,90.55,31.0


In [63]:
min_depth_dict = df_stats_full['RPC'].to_dict()

In [64]:
for depth in [50000, 45000, 40000, 35000, 30000, 25000, 20000, 15000, 10000, 5000]:
    min_value_str = 'k'.join(str(depth).rsplit('000', 1))
    if not os.path.exists(min_value_str):
        os.mkdir(min_value_str)
        os.mkdir(f"{min_value_str}/{min_value_str}_fastq")

In [65]:
downsample_dir = '/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series'
for depth in [50000, 45000, 40000, 35000, 30000, 25000, 20000, 15000, 10000, 5000]:
    parallel_filename = f"{depth}.downsample.parallel"
    
    min_value = depth

    olddepth = depth + 5000
    downsampling_depth = min_value/olddepth

    min_value_str = 'k'.join(str(min_value).rsplit('000', 1))
    olddepth_str = 'k'.join(str(olddepth).rsplit('000', 1))

    print(min_value_str)
    # print(olddepth_str)
    with open(parallel_filename, 'w') as f:
        for sample in sorted(df_stats_full.index):
            print(sample)
            if depth < min_depth_dict[sample]:
                if sample.split('_')[1] == "ddseq":
                    reads = [1, 2]
                else:
                    reads = [1, 2, 3]
                for read in reads:
                    if min_depth_dict[sample]-depth<=5000 or depth == 50000:
                        rate = depth/min_depth_dict[sample]
                        print(f"\t{rate}")
                        fastq = f"/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/full/{sample}__R{str(read)}.FULL.fastq.gz"
                        newfastq = f"/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/{min_value_str}/{min_value_str}_fastq/{sample}__R{str(read)}.{min_value_str}.fastq.gz"

                    else:
                        rate = depth/olddepth
                        print(f"\t{rate}")
                        fastq = f"/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/{olddepth_str}/{olddepth_str}_fastq/{sample}__R{str(read)}.{olddepth_str}.fastq.gz"
                        newfastq = f"/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/{min_value_str}/{min_value_str}_fastq/{sample}__R{str(read)}.{min_value_str}.fastq.gz"
                    if rate <= 1.0:
                        if not os.path.exists(newfastq):
                            f.write(f"seqtk sample -s 111 {fastq} {rate} | bgzip -@ 8 > {newfastq}\n")
                            print(f"\t\tseqtk sample -s 111 {fastq} {rate} | bgzip -@ 8 > {newfastq}")

                        else:
                            f.write(f"#{newfastq} already exists!\n")
                            print(f"\t\t#{newfastq} already exists!")

50k
BIO_ddseq_m1c1
BIO_ddseq_m1c2
BIO_ddseq_m1c3
BIO_ddseq_m1c4
BIO_ddseq_m1c5
BIO_ddseq_m1c6
BIO_ddseq_m1c7
BIO_ddseq_m1c8
BIO_ddseq_m2c1
	0.6341161098302904
		#/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/50k/50k_fastq/BIO_ddseq_m2c1__R1.50k.fastq.gz already exists!
	0.6341161098302904
		#/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/50k/50k_fastq/BIO_ddseq_m2c1__R2.50k.fastq.gz already exists!
BIO_ddseq_m2c2
BIO_ddseq_m2c3
	0.9774545654155682
		#/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/50k/50k_fastq/BIO_ddseq_m2c3__R1.50k.fastq.gz already exists!
	0.9774545654155682
		#/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/50k/50k_fastq/BIO_ddseq_m2c3__R2.50k.fastq.gz already exists!
BIO_ddseq_m2c4
	0.926832281333024
		#/lustre1/project/stg_00090/scatac_benchmark/public_downsample_series/50k/50k_fastq/BIO_ddseq_m2c4__R1.50k.fastq.gz already exists!
	0.926832281333024
		#/lustre1/project/stg_0009

Then invoke `parallel` as such:

```
my_conda_initialize
cat 50000.downsample.parallel | parallel -j 16 --progress
cat 45000.downsample.parallel | parallel -j 16 --progress
cat 40000.downsample.parallel | parallel -j 16 --progress
cat 35000.downsample.parallel | parallel -j 16 --progress
cat 30000.downsample.parallel | parallel -j 16 --progress
cat 25000.downsample.parallel | parallel -j 16 --progress
cat 20000.downsample.parallel | parallel -j 16 --progress
cat 15000.downsample.parallel | parallel -j 16 --progress
cat 10000.downsample.parallel | parallel -j 16 --progress
cat 5000.downsample.parallel | parallel -j 16 --progress
```

After running count and sort script:

In [70]:
!cat ./count_fastq_reads.sh

../0_resources/single_cell_toolkit_rust/target/release/fastq_count *k/*k_fastq/*R1*fastq.gz > R1_lengths.txt
../0_resources/single_cell_toolkit_rust/target/release/fastq_count *k/*k_fastq/*R2*fastq.gz > R2_lengths.txt
../0_resources/single_cell_toolkit_rust/target/release/fastq_count *k/*k_fastq/*R3*fastq.gz > R3_lengths.txt

sort R1_lengths.txt > R1_lengths.sorted.txt
sort R2_lengths.txt > R2_lengths.sorted.txt
sort R3_lengths.txt > R3_lengths.sorted.txt


In [71]:
read_count_df = pd.DataFrame()
for readfile in sorted(glob.glob("./R*_lengths.sorted.txt")):
    read = readfile.split("/")[-1].split("_")[0]
    df = pd.read_csv(readfile, sep="\t", header=None)
    df.index = [x.split("/")[-1].split(".fastq.gz")[0].replace('__R1','').replace('__R2','').replace('__R3','') for x in df[0]]
    df = df[1]
    read_count_df[read] = df

In [72]:
read_count_df

,R1,R2,R3
BIO_ddseq_m1c1.10k,42534343,42534343,NaN
BIO_ddseq_m1c2.10k,46874305,46874305,NaN
BIO_ddseq_m1c3.10k,43955875,43955875,NaN
BIO_ddseq_m1c4.10k,43725874,43725874,NaN
BIO_ddseq_m1c5.10k,45386536,45386536,NaN
...,...,...,...
VIB_hydrop_1.5k,16815363,16815363,16815363.0
VIB_hydrop_2.5k,10175331,10175331,10175331.0
VIB_hydrop_3.5k,13285679,13285679,13285679.0
VIB_hydrop_4.5k,11875590,11875590,11875590.0


In [73]:
read_dict = {}
for line in read_count_df.index:
    if read_count_df.isna().loc[line]["R3"]:
        if read_count_df.loc[line]["R1"] == read_count_df.loc[line]["R2"]:
            print(f"{line} OK")
            read_dict[line] = read_count_df.loc[line]["R1"]
        else:
            print(f"{line} has read discrepancy!")
            print(f'\t{read_count_df.loc[line]["R1"]}')
            print(f'\t{read_count_df.loc[line]["R2"]}')
    else:
        if (
            read_count_df.loc[line]["R1"]
            == read_count_df.loc[line]["R2"]
            == read_count_df.loc[line]["R3"]
        ):
            print(f"{line} OK")
            read_dict[line] = read_count_df.loc[line]["R1"]
        else:
            print(f"{line} has read discrepancy!")
            print(f'\t{read_count_df.loc[line]["R1"]}')
            print(f'\t{read_count_df.loc[line]["R2"]}')
            print(f'\t{read_count_df.loc[line]["R3"]}')

BIO_ddseq_m1c1.10k OK
BIO_ddseq_m1c2.10k OK
BIO_ddseq_m1c3.10k OK
BIO_ddseq_m1c4.10k OK
BIO_ddseq_m1c5.10k OK
BIO_ddseq_m1c6.10k OK
BIO_ddseq_m1c7.10k OK
BIO_ddseq_m1c8.10k OK
BIO_ddseq_m2c1.10k OK
BIO_ddseq_m2c2.10k OK
BIO_ddseq_m2c3.10k OK
BIO_ddseq_m2c4.10k OK
OHS_s3atac_mouse.10k OK
TXG_10xmultiome_e18mousebrainfresh.10k OK
TXG_10xv11_adultmousecortexchromiumx.10k OK
TXG_10xv1_adultmousefresh.10k OK
TXG_10xv2_adultmousecortex.10k OK
TXG_10xv2_adultmousecortexchromiumx.10k OK
VIB_hydrop_1.10k OK
VIB_hydrop_2.10k OK
VIB_hydrop_3.10k OK
VIB_hydrop_4.10k OK
VIB_hydrop_5.10k OK
BIO_ddseq_m1c1.15k OK
BIO_ddseq_m1c2.15k OK
BIO_ddseq_m1c3.15k OK
BIO_ddseq_m1c4.15k OK
BIO_ddseq_m1c5.15k OK
BIO_ddseq_m1c6.15k OK
BIO_ddseq_m1c7.15k OK
BIO_ddseq_m1c8.15k OK
BIO_ddseq_m2c1.15k OK
BIO_ddseq_m2c2.15k OK
BIO_ddseq_m2c3.15k OK
BIO_ddseq_m2c4.15k OK
OHS_s3atac_mouse.15k OK
TXG_10xmultiome_e18mousebrainfresh.15k OK
TXG_10xv11_adultmousecortexchromiumx.15k OK
TXG_10xv1_adultmousefresh.15k OK
TXG_10xv2

# Determine cell counts

In [76]:
tsv_list = sorted(
    glob.glob(f"../public_3_cistopic_qc/selected_barcodes/*.FULL_bc_passing_filters_otsu.txt")
)
cell_count_dict = {}
for tsv in tsv_list:
    sample = tsv.split("/")[-1].split(".")[0].split("_metadata_bc_df")[0]
    # print(sample)
    df = pd.read_csv(tsv, sep="\t", index_col=0)
    cell_count_dict[sample] = len(df)

In [77]:
cell_count_dict

{'BIO_ddseq_m1c1': 4253,
 'BIO_ddseq_m1c2': 4687,
 'BIO_ddseq_m1c3': 4395,
 'BIO_ddseq_m1c4': 4372,
 'BIO_ddseq_m1c5': 4538,
 'BIO_ddseq_m1c6': 4579,
 'BIO_ddseq_m1c7': 3726,
 'BIO_ddseq_m1c8': 3273,
 'BIO_ddseq_m2c1': 2536,
 'BIO_ddseq_m2c2': 3456,
 'BIO_ddseq_m2c3': 2416,
 'BIO_ddseq_m2c4': 2300,
 'OHS_s3atac_mouse': 956,
 'TXG_10xmultiome_e18mousebrainfresh': 4514,
 'TXG_10xv11_adultmousecortexchromiumx': 6089,
 'TXG_10xv1_adultmousefresh': 3656,
 'TXG_10xv2_adultmousecortex': 6395,
 'TXG_10xv2_adultmousecortexchromiumx': 6623,
 'VIB_hydrop_1': 3362,
 'VIB_hydrop_2': 2034,
 'VIB_hydrop_3': 2656,
 'VIB_hydrop_4': 2374,
 'VIB_hydrop_5': 2058}

Now, calculate the RPC:

In [78]:
read_count_df['sample'] = [x.split('.')[0] for x in read_count_df.index]

In [79]:
read_count_df['n_cells'] = [cell_count_dict[x] for x in read_count_df['sample']]

In [80]:
read_count_df['RPC'] = read_count_df['R1']/read_count_df['n_cells']

In [81]:
read_count_df

,R1,R2,R3,sample,n_cells,RPC
BIO_ddseq_m1c1.10k,42534343,42534343,NaN,BIO_ddseq_m1c1,4253,10001.021162
BIO_ddseq_m1c2.10k,46874305,46874305,NaN,BIO_ddseq_m1c2,4687,10000.918498
BIO_ddseq_m1c3.10k,43955875,43955875,NaN,BIO_ddseq_m1c3,4395,10001.336746
BIO_ddseq_m1c4.10k,43725874,43725874,NaN,BIO_ddseq_m1c4,4372,10001.343550
BIO_ddseq_m1c5.10k,45386536,45386536,NaN,BIO_ddseq_m1c5,4538,10001.440282
...,...,...,...,...,...,...
VIB_hydrop_1.5k,16815363,16815363,16815363.0,VIB_hydrop_1,3362,5001.595181
VIB_hydrop_2.5k,10175331,10175331,10175331.0,VIB_hydrop_2,2034,5002.620944
VIB_hydrop_3.5k,13285679,13285679,13285679.0,VIB_hydrop_3,2656,5002.138178
VIB_hydrop_4.5k,11875590,11875590,11875590.0,VIB_hydrop_4,2374,5002.354676


In [82]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(read_count_df['RPC'])

BIO_ddseq_m1c1.10k                          10001.021
BIO_ddseq_m1c2.10k                          10000.918
BIO_ddseq_m1c3.10k                          10001.337
BIO_ddseq_m1c4.10k                          10001.344
BIO_ddseq_m1c5.10k                          10001.440
BIO_ddseq_m1c6.10k                          10000.883
BIO_ddseq_m1c7.10k                          10001.725
BIO_ddseq_m1c8.10k                          10002.044
BIO_ddseq_m2c1.10k                          10002.693
BIO_ddseq_m2c2.10k                          10001.841
BIO_ddseq_m2c3.10k                          10003.434
BIO_ddseq_m2c4.10k                          10003.888
OHS_s3atac_mouse.10k                        10006.857
TXG_10xmultiome_e18mousebrainfresh.10k      10001.309
TXG_10xv11_adultmousecortexchromiumx.10k    10000.549
TXG_10xv1_adultmousefresh.10k               10002.100
TXG_10xv2_adultmousecortex.10k              10000.389
TXG_10xv2_adultmousecortexchromiumx.10k     10000.248
VIB_hydrop_1.10k            

Looks good.